In [ ]:
import pickle

import keras.preprocessing.sequence as k_ps
import keras.models as k_mod
import keras.layers as k_lay
import keras.optimizers as k_opt
import keras.utils as k_ut
import numpy as np
import pandas as pd
import sklearn.metrics as sk_mt
import sklearn.model_selection as sk_ms
import sklearn.preprocessing as sk_pp
import sklearn.utils as sk_ut

In [ ]:
with open('lstm_tweets_net.pkl', 'rb') as f:
    data = pickle.load(f)
x, y = zip(*data)

In [ ]:
pad_tag = '#PAD#'
max_len = max(len(xw) for xw in x)

In [ ]:
flat_tmp_x = set([b for a in x for b in a])

indexed_tmp_x = {w: i + 1 for i, w in enumerate(flat_tmp_x)}
indexed_tmp_x[pad_tag] = 0
tagged_x = np.array([[indexed_tmp_x[wx] for wx in sx] for sx in x])

lb = sk_pp.LabelBinarizer()
lb.fit(y)

x_tagged_train, x_tagged_test, y_train, y_test = sk_ms.train_test_split(tagged_x, lb.transform(y), shuffle=True)

In [ ]:
input_dim = len(x[0])
output_dim = len(set(y))

In [ ]:
model = k_mod.Sequential()
model.add(k_lay.Embedding(len(flat_tmp_x) + 1, 128, input_length=max_len))
model.add(k_lay.LSTM(128))
model.add(k_lay.Dense(output_dim, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=k_opt.Adam(0.001), metrics=['accuracy'])
model.fit(x_tagged_train, y_train, batch_size=128, epochs=10)

In [ ]:
pred = model.predict_classes(x_tagged_test)
y_pred = lb.inverse_transform(k_ut.to_categorical(pred))

In [ ]:
print(sk_mt.classification_report(lb.inverse_transform(y_test), y_pred))